<a href="https://colab.research.google.com/github/superbunny38/2021DeepLearning/blob/main/pytorch/Chap_8_(2)_Inception.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from glob import glob
import os
from shutil import copyfile
from torch.utils.data import Dataset
from PIL import Image
import numpy as np
from numpy.random import permutation
import matplotlib.pyplot as plt

from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision.models.inception import inception_v3
from torch.utils.data import DataLoader
from torch.autograd import Variable
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

%matplotlib inline

In [2]:
is_cuda = torch.cuda.is_available()
print(is_cuda)
print(torch.cuda.memory_allocated())
torch.cuda.empty_cache()

True
0


## Utility functions

In [23]:
def imshow(inp,cmap=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp,cmap)
    
class LayerActivations():
    features=[]
    
    def __init__(self,model):
        self.features = []
        self.hook = model.register_forward_hook(self.hook_function)
    
    def hook_function(self,module,input,output):
        
        self.features.extend(output.view(output.size(0),-1).cpu().data)

    
    def remove(self):
        
        self.hook.remove()
    
class FeaturesDataset(Dataset):
    
    def __init__(self,featlst,labellst):
        self.featlst = featlst
        self.labellst = labellst
        
    def __getitem__(self,index):
        return (self.featlst[index],self.labellst[index])
    
    def __len__(self):
        return len(self.labellst)
    
def fit(epoch,model,data_loader,phase='training',volatile=False):
    if phase == 'training':
        model.train()
    if phase == 'validation':
        model.eval()
        volatile=True
    running_loss = 0.0
    running_correct = 0
    for batch_idx , (data,target) in enumerate(data_loader):
        if is_cuda:
            data,target = data.cuda(),target.cuda()
        data , target = Variable(data,volatile),Variable(target)
        if phase == 'training':
            optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output,target)
        
        running_loss += F.cross_entropy(output,target,size_average=False).data
        preds = output.data.max(dim=1,keepdim=True)[1]
        running_correct += preds.eq(target.data.view_as(preds)).cpu().sum()
        if phase == 'training':
            loss.backward()
            optimizer.step()
    
    loss = running_loss/len(data_loader.dataset)
    accuracy = 100. * running_correct/len(data_loader.dataset)
    
    print(f'{phase} loss is {loss} and {phase} accuracy is {running_correct}/{len(data_loader.dataset)}{accuracy}')
    return loss,accuracy

## get DATA

In [5]:
!git clone https://github.com/ardamavi/Dog-Cat-Classifier repo
!mkdir workspace
%mv /content/repo/Data/Train_Data /content/workspace
%cd workspace

Cloning into 'repo'...
remote: Enumerating objects: 1654, done.
remote: Total 1654 (delta 0), reused 0 (delta 0), pack-reused 1654
Receiving objects: 100% (1654/1654), 34.83 MiB | 30.88 MiB/s, done.
Resolving deltas: 100% (147/147), done.
/content/workspace


In [6]:
import glob
path = "/content/workspace/Train_Data"
#read all the files inside the folder
dog_files = [f for f in glob.glob("/content/workspace/Train_Data/dog/*.jpg")]
cat_files = [f for f in glob.glob("/content/workspace/Train_Data/cat/*.jpg")]

files = dog_files+cat_files
print(f"Total number of images: {len(files)}")
no_of_images = len(files)
shuffle = np.random.permutation(no_of_images)
os.mkdir(os.path.join(path, "train"))
os.mkdir(os.path.join(path, "valid"))
for t in ['train','valid']:
  for folder in ['dog/', 'cat/']:
    os.mkdir(os.path.join(path,t,folder))

Total number of images: 1399


In [7]:
#검증데이터 넣기
for i in shuffle[:250]:#250개
  folder = files[i].split("/")[-2].split(".")[0]
  image = files[i].split('/')[-1]
  image = image[4:]
  os.rename(files[i], os.path.join(path, 'valid', folder, image))

In [8]:
#학습데이터 넣기
for i in shuffle[250:]:
  folder = files[i].split("/")[-2].split(".")[0]
  image = files[i].split('/')[-1]
  image = image[4:]
  os.rename(files[i], os.path.join(path, 'train', folder, image))

In [11]:
transform_data = transforms.Compose([
        transforms.Resize((299,299)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

## create dataloaders

In [12]:
from torchvision.datasets import ImageFolder
train_dset = ImageFolder("/content/workspace/Train_Data/train", transform_data)
validation_dset = ImageFolder("/content/workspace/Train_Data/valid", transform_data)
classes=2

In [13]:
train_loader = DataLoader(train_dset,batch_size=32,shuffle=False,num_workers=3)
validation_loader = DataLoader(validation_dset,batch_size=32,shuffle=False,num_workers=3)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


## Creating Inception V3 model

In [14]:
inception_model = inception_v3(pretrained=True)
inception_model.aux_logits = False
if is_cuda:
    inception_model = inception_model.cuda()

Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth


  0%|          | 0.00/104M [00:00<?, ?B/s]

## Extracting Convolutional features

In [15]:
# Create LayerActivations object to store the output of inception model at a particular layer.
is_cuda = torch.cuda.is_available()
print(is_cuda)
train_features = LayerActivations(inception_model.Mixed_7c)
train_labels = []

# Passing all the data through the model , as a side effect the outputs will get stored 
# in the features list of the LayerActivations object. 
for da,la in train_loader:
    with torch.no_grad(): # To help with memory issues
        _ = inception_model(Variable(da.cuda()))
        train_labels.extend(la.cpu())
train_features.remove()

print('Done')

# Repeat the same process for validation dataset .

validation_features = LayerActivations(inception_model.Mixed_7c)
validation_labels = []
for da,la in validation_loader:
    with torch.no_grad(): # To help with memory issues
        _ = inception_model(Variable(da.cuda()))
        validation_labels.extend(la.cpu())
validation_features.remove()

True


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Done


In [16]:
len(validation_labels)

250

In [17]:
train_features.features[0].size()

torch.Size([131072])

In [18]:
len(train_features.features)

1149

## Creating train and validation feature dataset

In [19]:
#Dataset for pre computed features for train and validation data sets

train_feat_dset = FeaturesDataset(train_features.features,train_labels)
validation_feat_dset = FeaturesDataset(validation_features.features,validation_labels)

#Data loaders for pre computed features for train and validation data sets

train_feat_loader = DataLoader(train_feat_dset,batch_size=64,shuffle=True)
validation_feat_loader = DataLoader(validation_feat_dset,batch_size=64)

## A Fully connected network

In [20]:
class FullyConnectedModel(nn.Module):
    
    def __init__(self,input_size,output_size,training=True):
        super().__init__()
        self.fully_connected = nn.Linear(input_size,output_size)

    def forward(self,input):
        output = F.dropout(input, training=self.training)
        output = self.fully_connected(output)
        return output

# The size of the output from the selected convolution feature 
fc_in_size = 131072

fully_connected = FullyConnectedModel(fc_in_size,classes)
if is_cuda:
    fully_connected = fully_connected.cuda()

In [21]:
optimizer = optim.Adam(fully_connected.parameters(),lr=0.01)

## Train and validate the model

In [24]:
train_losses , train_accuracy = [],[]
validation_losses , validation_accuracy = [],[]
for epoch in range(1,10):
    epoch_loss, epoch_accuracy = fit(epoch,fully_connected,train_feat_loader,phase='training')
    validation_epoch_loss , validation_epoch_accuracy = fit(epoch,fully_connected.eval(),validation_feat_loader,phase='validation')
    train_losses.append(epoch_loss)
    train_accuracy.append(epoch_accuracy)
    validation_losses.append(validation_epoch_loss)
    validation_accuracy.append(validation_epoch_accuracy)

/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


training loss is 63.613826751708984 and training accuracy is 660/114957.441253662109375
validation loss is 18.748445510864258 and validation accuracy is 170/25068.0
training loss is 5.514293193817139 and training accuracy is 998/114986.85813903808594
validation loss is 4.40522575378418 and validation accuracy is 221/25088.4000015258789
training loss is 1.0712841749191284 and training accuracy is 1104/114996.08354949951172
validation loss is 3.1412394046783447 and validation accuracy is 222/25088.80000305175781
training loss is 0.36424702405929565 and training accuracy is 1129/114998.25935363769531
validation loss is 3.1379079818725586 and validation accuracy is 224/25089.5999984741211
training loss is 0.27743473649024963 and training accuracy is 1130/114998.34638977050781
validation loss is 3.1160669326782227 and validation accuracy is 226/25090.4000015258789
training loss is 0.10593618452548981 and training accuracy is 1140/114999.21671295166016
validation loss is 5.919647693634033 an

In [25]:
for epoch in range(1,10):
    epoch_loss, epoch_accuracy = fit(epoch,fully_connected,train_feat_loader,phase='training')
    validation_epoch_loss , validation_epoch_accuracy = fit(epoch,fully_connected,validation_feat_loader,phase='validation')
    train_losses.append(epoch_loss)
    train_accuracy.append(epoch_accuracy)
    validation_losses.append(validation_epoch_loss)
    validation_accuracy.append(validation_epoch_accuracy)

/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


training loss is 0.03614313155412674 and training accuracy is 1147/114999.82593536376953
validation loss is 4.304982662200928 and validation accuracy is 219/25087.5999984741211
training loss is 0.03944215551018715 and training accuracy is 1145/114999.65187072753906
validation loss is 2.9763283729553223 and validation accuracy is 226/25090.4000015258789
training loss is 0.0022036107257008553 and training accuracy is 1148/114999.91297149658203
validation loss is 3.6348624229431152 and validation accuracy is 225/25090.0
training loss is 0.017398081719875336 and training accuracy is 1148/114999.91297149658203
validation loss is 4.344318866729736 and validation accuracy is 222/25088.80000305175781
training loss is 0.06886065751314163 and training accuracy is 1143/114999.4778060913086
validation loss is 3.8241934776306152 and validation accuracy is 224/25089.5999984741211
training loss is 0.09919421374797821 and training accuracy is 1142/114999.39077758789062
validation loss is 8.15740966796